# Let's look at the raw data, specifically the washover stats

In [1]:
# Import packages we need

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os,shutil

In [2]:
#load the labels
#round 2 labels
#labeler_data = pd.read_csv('../data/tables/labels/Priority_1604952200293.csv')
#round 3 labels
labeler_data = pd.read_csv('../data/tables/labelerdata/Priority_1613396170162.csv')

## Get data into shape for release

In [3]:
# make data into final zenodo form
releaseData = labeler_data
releaseData = releaseData.drop(['_id','catalogId', 'archiveId','date','catalog','additionalComments'], axis = 1)
releaseData = releaseData.drop(['imageId'], axis = 1)
releaseData = releaseData.rename({'water': 'allWater'}, axis=1)
releaseData.head()

,userId,archive,image,allWater,devType,washoverType,dmgType,impactType,terrianType
0,5f4c002a9cdb9b001675a8e2,Florence,P26058806.jpg,True,NaN,NaN,NaN,NaN,NaN
1,5f4c002a9cdb9b001675a8e2,Michael,S28468531.jpg,NaN,developed,noWashover,noDamage,NaN,"marsh,inland,river"
2,5f4c002a9cdb9b001675a8e2,Michael,S28466899.jpg,NaN,developed,noWashover,damage,NaN,inland
3,5f4c002a9cdb9b001675a8e2,Isaias,S21773987.jpg,NaN,developed,noWashover,noDamage,swash,sandyCoastline
4,5f4c002a9cdb9b001675a8e2,Florence,P26057665.jpg,NaN,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline


In [4]:
# replace NaN in devType, washoverType, dmgType,
values = {'allWater':'False', 'devType': 'undeveloped', 'washoverType': 'noWashover', 'dmgType': 'noDamage'}

releaseData = releaseData.fillna(value=values)
releaseData

,userId,archive,image,allWater,devType,washoverType,dmgType,impactType,terrianType
0,5f4c002a9cdb9b001675a8e2,Florence,P26058806.jpg,True,undeveloped,noWashover,noDamage,NaN,NaN
1,5f4c002a9cdb9b001675a8e2,Michael,S28468531.jpg,False,developed,noWashover,noDamage,NaN,"marsh,inland,river"
2,5f4c002a9cdb9b001675a8e2,Michael,S28466899.jpg,False,developed,noWashover,damage,NaN,inland
3,5f4c002a9cdb9b001675a8e2,Isaias,S21773987.jpg,False,developed,noWashover,noDamage,swash,sandyCoastline
4,5f4c002a9cdb9b001675a8e2,Florence,P26057665.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline
...,...,...,...,...,...,...,...,...,...
4577,5ff24c3448a4570013d1a9ed,IsaiasWash3,P21772651.jpg,False,developed,noWashover,noDamage,swash,sandyCoastline
4578,5ff24c3448a4570013d1a9ed,IsaiasWash3,S21771105.jpg,False,undeveloped,noWashover,noDamage,swash,"sandyCoastline,marsh"
4579,5ff24c3448a4570013d1a9ed,IsaiasWash3,S21771797.jpg,False,developed,noWashover,noDamage,NaN,inland
4580,5ff24c3448a4570013d1a9ed,IsaiasWash3,C21768037.jpg,False,developed,noWashover,noDamage,NaN,"marsh,inland"


In [5]:
releaseData.shape

(4582, 9)

In [6]:
Threshold = releaseData[["userId","archive","image"]].groupby(['userId', 'archive']).count().reset_index()
#Threshold = Threshold[Threshold["image"] == 100].reset_index()
Threshold

,userId,archive,image
0,5f4c002a9cdb9b001675a8e2,Florence,100
1,5f4c002a9cdb9b001675a8e2,Florence3,100
2,5f4c002a9cdb9b001675a8e2,FlorenceWash3,100
3,5f4c002a9cdb9b001675a8e2,Isaias,100
4,5f4c002a9cdb9b001675a8e2,Isaias3,100
5,5f4c002a9cdb9b001675a8e2,IsaiasWash3,100
6,5f4c002a9cdb9b001675a8e2,Michael,100
7,5f4c002a9cdb9b001675a8e2,Michael3,100
8,5f4c002a9cdb9b001675a8e2,MichaelWash3,100
9,5f4f13135463f70016a500ed,Florence,1


In [7]:
#add a true column
Threshold.rename(columns = {"image": "total"}, inplace=True)

#join Threshold with releaseData
releaseData = releaseData.merge(Threshold, left_on = ['userId', 'archive'], right_on = ['userId', 'archive'])

# remove rows where total <100
releaseData = releaseData[releaseData["total"] == 100]
releaseData.head()

,userId,archive,image,allWater,devType,washoverType,dmgType,impactType,terrianType,total
0,5f4c002a9cdb9b001675a8e2,Florence,P26058806.jpg,True,undeveloped,noWashover,noDamage,NaN,NaN,100
1,5f4c002a9cdb9b001675a8e2,Florence,P26057665.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
2,5f4c002a9cdb9b001675a8e2,Florence,P26059250.jpg,False,developed,washover,noDamage,"swash,collision,overwash",sandyCoastline,100
3,5f4c002a9cdb9b001675a8e2,Florence,P26060067.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
4,5f4c002a9cdb9b001675a8e2,Florence,P26057539.jpg,False,undeveloped,washover,noDamage,"swash,collision,overwash",sandyCoastline,100


In [8]:
#replace user IDs with `User 1`, etc...
releaseData.shape
releaseData.userId.unique()

array(['5f4c002a9cdb9b001675a8e2', '5f77871024f95e0018305349',
       '5f6635628cb36a001698427b', '5f7af56443360b001790063d',
       '5f7c8c4287bff50017610126', '5f89a4957c0f40001652c05d',
       '5f89c25b67d0e30016a5e70a', '5fbc7ccabf106c0017b864a0',
       '5fc7dc859f59bc0016211439', '5f8f5f59e4dac20017f9070e',
       '600994de5eff87001694a117', '5fa4542fec49d20016f772a8',
       '5ff24c3448a4570013d1a9ed'], dtype=object)

In [9]:
releaseData['userId'] = pd.factorize(releaseData.userId)[0] + 1
releaseData.userId.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [10]:
releaseData.head()

,userId,archive,image,allWater,devType,washoverType,dmgType,impactType,terrianType,total
0,1,Florence,P26058806.jpg,True,undeveloped,noWashover,noDamage,NaN,NaN,100
1,1,Florence,P26057665.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
2,1,Florence,P26059250.jpg,False,developed,washover,noDamage,"swash,collision,overwash",sandyCoastline,100
3,1,Florence,P26060067.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
4,1,Florence,P26057539.jpg,False,undeveloped,washover,noDamage,"swash,collision,overwash",sandyCoastline,100


In [11]:
#replace column ittle with NOAA flight
releaseData = releaseData.rename({'archive': 'NOAA_flight'}, axis=1)

# replace archive with specific flight
releaseData = releaseData.replace('Florence','Florence_20180917a_jpgs')
releaseData = releaseData.replace('Michael', 'Michael_20181011a_jpgs')
releaseData = releaseData.replace('Isaias','Isaias_20200804a_jpgs')

# replace archive with specific flight
releaseData = releaseData.replace('Florence3','Florence_20180917a_jpgs')
releaseData = releaseData.replace('Michael3', 'Michael_20181011a_jpgs')
releaseData = releaseData.replace('Isaias3','Isaias_20200804a_jpgs')

# replace archive with specific flight
releaseData = releaseData.replace('FlorenceWash3','Florence_20180917a_jpgs')
releaseData = releaseData.replace('MichaelWash3', 'Michael_20181011a_jpgs')
releaseData = releaseData.replace('IsaiasWash3','Isaias_20200804a_jpgs')

releaseData.head()

,userId,NOAA_flight,image,allWater,devType,washoverType,dmgType,impactType,terrianType,total
0,1,Florence_20180917a_jpgs,P26058806.jpg,True,undeveloped,noWashover,noDamage,NaN,NaN,100
1,1,Florence_20180917a_jpgs,P26057665.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
2,1,Florence_20180917a_jpgs,P26059250.jpg,False,developed,washover,noDamage,"swash,collision,overwash",sandyCoastline,100
3,1,Florence_20180917a_jpgs,P26060067.jpg,False,undeveloped,noWashover,noDamage,"swash,collision",sandyCoastline,100
4,1,Florence_20180917a_jpgs,P26057539.jpg,False,undeveloped,washover,noDamage,"swash,collision,overwash",sandyCoastline,100


In [12]:
#save release data
releaseData.to_csv('../data/tables/labelerdata/ReleaseData2.csv',na_rep='NaN')